In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sydney,-33.8679,151.2073,20.17,92,75,9.77,AU,1705259076
1,1,klaksvik,62.2266,-6.5890,-2.49,82,100,17.19,FO,1705259893
2,2,adamstown,-25.0660,-130.1015,23.94,76,1,6.66,PN,1705259893
3,3,prado,-17.3411,-39.2208,28.35,71,7,6.20,BR,1705259893
4,4,bredasdorp,-34.5322,20.0403,19.96,71,66,2.19,ZA,1705259863


Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
# Configure the map plot
humidity_map = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity', 
                                          color='City', hover_cols=['City', 'Country','Humidity'], 
                                          tiles='OSM', frame_width=700, frame_height=500)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_conditions = (city_data_df['Max Temp'] < 24) & (city_data_df['Max Temp'] > 12) & (city_data_df['Wind Speed'] < 3.5) & (city_data_df['Cloudiness'] == 0)
ideal_cities_df = city_data_df[ideal_weather_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,lihue,21.9789,-159.3672,21.58,68,0,2.57,US,1705259857
67,67,margaret river,-33.9500,115.0667,20.48,79,0,3.10,AU,1705259857
97,97,wailua homesteads,22.0669,-159.3780,21.24,67,0,2.57,US,1705259865
212,212,ascension,30.2169,-90.9001,19.44,44,0,3.09,US,1705259915
229,229,la'ie,21.6477,-157.9253,23.36,72,0,1.34,US,1705259862
269,269,santa rosalia,27.3167,-112.2833,21.75,29,0,1.73,MX,1705259923
272,272,jamnagar,22.4667,70.0667,20.56,54,0,3.03,IN,1705259924
301,301,kapa'a,22.0752,-159.3190,21.99,69,0,2.57,US,1705259714
302,302,new kingman-butler,35.2650,-114.0323,12.33,24,0,3.09,US,1705259930
313,313,holualoa,19.6228,-155.9522,21.91,71,0,0.00,US,1705259934


Step 3: Create a new DataFrame called hotel_df.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,lihue,US,21.9789,-159.3672,68,
67,margaret river,AU,-33.9500,115.0667,79,
97,wailua homesteads,US,22.0669,-159.3780,67,
212,ascension,US,30.2169,-90.9001,44,
229,la'ie,US,21.6477,-157.9253,72,
269,santa rosalia,MX,27.3167,-112.2833,29,
272,jamnagar,IN,22.4667,70.0667,54,
301,kapa'a,US,22.0752,-159.3190,69,
302,new kingman-butler,US,35.2650,-114.0323,24,
313,holualoa,US,19.6228,-155.9522,71,


Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.